In [1]:
! pip install selenium

In [2]:
! pip install webdriver_manager

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import datetime
import pprint as pp
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = "https://www.boamp.fr/pages/entreprise-accueil/"
today = datetime.date.today().strftime("%d/%m/%Y")

In [15]:
options = Options()
# options.add_argument("--headless")  # Run in headless mode
# options.binary_location ="/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options) #driver_version="137.0.7151.122"
driver.get(url)

#recherche des AO aujourd'hui
time.sleep(3)
driver.find_element(By.XPATH, './/a[@class="fr-btn"]').click()
time.sleep(3)
driver.find_element(By.XPATH, './/button[@class="fr-accordion__btn ng-binding"]').click()
time.sleep(5)
driver.find_element(By.XPATH, './/div[@class="odswidget-timerange__from"]/input').send_keys(today)
time.sleep(3)
driver.find_element(By.XPATH, './/div[@class="odswidget-timerange__to"]/input').send_keys(today)
time.sleep(5)
driver.find_element(By.XPATH, './/a[@class="fr-btn"]').click()
time.sleep(3)



In [16]:
import math
#calcul nombre pages
nb_pages = math.ceil(int(driver.find_element(By.XPATH, './/p[@class="fr-m-0 fr-text--lg ng-scope"]').text[:-30])/10)
nb_pages

11

In [17]:
j=5
liens=[]
#liste de liens d'appels d'offres
for i in range (1, 10):
    if i > 6 and i != nb_pages-5:
        # print(i)
        driver.find_element(By.XPATH, './/div[@class="fr-container no-print"]/nav/ul/li[6]').click()
        time.sleep(5)
        liens.extend([lien.get_attribute('href') for lien in driver.find_elements(By.XPATH, './/a[@class="fr-btn fr-my-1w ng-binding"]')])
    elif i == nb_pages-5:
        # print(i)
        j = j + 1
        driver.find_element(By.XPATH, './/div[@class="fr-container no-print"]/nav/ul/li[' + str(j) + ']').click()
        time.sleep(5)
        liens.extend([lien.get_attribute('href') for lien in driver.find_elements(By.XPATH, './/a[@class="fr-btn fr-my-1w ng-binding"]')])
    else:
        # print(i)
        driver.find_element(By.XPATH, './/ul[@class="no-list-style ng-scope"]/li[' + str(i) + ']').click()
        time.sleep(10)
        liens.extend([lien.get_attribute('href') for lien in driver.find_elements(By.XPATH, './/a[@class="fr-btn fr-my-1w ng-binding"]')]) # les liens
driver.quit()

In [21]:
AO=[]
open('demo.log', 'w').close()  # Clear the log file
for lien in liens[:5]:
        if lien in open('demo.log').read():
            print("Appel d'offre déjà traité" )
        else :
            options = Options()
            options.add_argument('--disable-blink-features=AutomationControlled')
            options.add_argument('--headless')  # Run in headless mode
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options) # driver_version="137.0.7151.122"
            driver.get(lien)
            time.sleep(5)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[@class="fr-btn fr-my-1w ng-binding"]'))).click()
            # driver.find_element(By.XPATH, './/button[@class="fr-btn fr-my-1w ng-binding"]').click()
            time.sleep(10)
            text_ao = driver.find_element(By.XPATH, './/div[@class="ng-scope"]').text
            driver.quit()
            AO.append(text_ao[text_ao.find("Avis n°"):text_ao.find("Date d'envoi du présent avis à la publication")])
            with open ('demo.log', 'a') as f:
                f.write("Appel d'offre : " + lien + " traité\n")
        #driver.quit()
print("Appels d'offres récupérés !")

Appels d'offres récupérés !


In [21]:
# driver.quit()

In [ ]:
# AO = []
# #AO liste = texte "clean" des X appels d'offres
# for lien in liens:
#     driver.get(lien)
#     time.sleep(5)
#     driver.find_element(By.XPATH, './/button[@class="fr-btn fr-my-1w ng-binding"]').click()
#     time.sleep(10)
#     text_ao = driver.find_element(By.XPATH, './/div[@class="ng-scope"]').text
#     AO.append(text_ao[text_ao.find("Avis n°"):text_ao.find("Date d'envoi du présent avis à la publication")])
    

In [250]:
print(AO[0])

Avis n° 25-70875
Détail de l'avis 25-70875
Avis n° 25-70875
Publié le 24 juin 2025
Date limite de réponse le 21/07/2025 à 12h00
Fourniture et livraison de mobiliers urbains pour la Ville de Coudekerque-Branche
DÉPARTEMENT : 59
ACHETEUR : VILLE DE COUDEKERQUE BRANCHE
TYPE D'AVIS : Avis de marché
PROCÉDURE : Procédure Ouverte
Références complètes : Avis n° 25-70875
Cacher le contenu de l'annonce
Obtenir un extrait de l’avis (PDF)
Avis de marché
Département(s) de publication : 59
Annonce n° 25-70875
Fournitures
Section 1 - Identification de l'acheteur
Nom complet de l'acheteur : VILLE DE COUDEKERQUE BRANCHE
Type de Numéro national d'indentification : SIRET
N° National d'identification : 21590155400018
Ville : COUDEKERQUE BRANCHE CEDEX
Code postal : 59411
Groupement de commandes : Non
Département(s) de publication : 59
Section 2 - Communication
Lien vers le profil d'acheteur : https://www.marches-securises.fr/
Identifiant interne de la consultation : 2025.08
Intégralité des documents sur l

In [251]:
with open("AO.txt", "w") as f:
    for avis in AO:
        f.write(avis + "\n\n")

In [23]:
driver.quit()